In [3]:
import numpy as np
import sys

In [66]:
f = open('./dc.in', 'r')

farm_rows = -1
farm_slots = -1 
slots_unav = 10000000 
pools = -1 
servers = -1 

slots_unav_idx = []
servers_info = []

for i, each in enumerate(f.readlines()):
    if i == 0:
        farm_rows  =int(each.split(' ')[0])
        farm_slots = int(each.split(' ')[1])
        slots_unav = int(each.split(' ')[2])
        pools = int(each.split(' ')[3])
        servers = int(each.split(' ')[4])
    else:
        if i <= slots_unav:
            row_tmp = int(each.split(' ')[0])
            slot_tmp = int(each.split(' ')[1])
            slots_unav_idx.append([row_tmp, slot_tmp])

        if i > slots_unav:
            slots_tmp = int(each.split(' ')[0])
            capacity_tmp = int(each.split(' ')[1])
            servers_info.append([slots_tmp, capacity_tmp])
        
    # if i > 200: break

print ('Farm Size : ', farm_rows, farm_slots)
print ('Unavailable : ', slots_unav)
# print ('Unavailable : ', slots_unav_idx)
print ('Pools : ', pools, ' || Servers : ', servers)


Farm Size :  16 100
Unavailable :  80
Pools :  45  || Servers :  625


In [81]:
farm = np.zeros((farm_rows, farm_slots), dtype=np.int16)

for each in slots_unav_idx:
    row_tmp   = each[0]
    slots_tmp = each[1]
    farm[row_tmp][slot_tmp] = -1
    print (row_tmp, slots_tmp, farm[row_tmp][slot_tmp])
    print (farm[row_tmp])

for each in farm:
    print (each)

10 23 -1
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0 -1  0]
0 40 -1
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0 -1  0]
4 40 -1
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0 -1  0]
3 57 -1
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [70]:
farm = np.zeros((farm_rows, farm_slots), dtype=np.int16)

for each in slots_unav_idx:
    row_tmp = each[0]
    slots_tmp = each[1]
    farm[row_tmp][slot_tmp] = -1

servers_info = sorted(servers_info, key=lambda x: x[1])[::-1]

tmp  = np.array(servers_info)
tmp2 = np.zeros((tmp.shape[0], tmp.shape[1]+1))
tmp2[:,0:2]  = tmp
tmp2[:,2]    = tmp[:,1]/tmp[:,0]
tmp2         = sorted(tmp2, key=lambda x: x[2])[::-1]

servers_info2 = list(servers_info)
for i, each in enumerate(tmp2):
    servers_info2[i] = tmp2[i].tolist()

###################################################################

def isSlotAvailable (row, server, server_id):
    rowLen = len (row)
    serverSlots = server[0]
    zeroCount = 0
    
    for i in range (rowLen):
        if (zeroCount >= serverSlots):
            return (i-zeroCount, i) # Look for the edge case
        if (row[i] == 0):
            zeroCount += 1
        elif (row[i] == -1):
            zeroCount = 0
        elif (row[i] >= 1):
            zeroCount = 0
        else:
            print ("Not sure why the loop came here")
            print (row, i)
            sys.exit(1)

    return (-1, -1)

server_dict = {}
farm_row_tmp = 0
    
for server_id, server in enumerate(servers_info2):
    server_slots    = server[0]
    server_capacity = server[1]
    
    trials = 0
    while (1):
        startSlot, endSlot = isSlotAvailable (farm[farm_row_tmp], server, server_id)
        if (startSlot == -1):
            if trials > farm_rows:
                print ('Failed at server_id : ', server_id, server[0])
                break
            farm_row_tmp = (farm_row_tmp + 1) % (farm_rows)
            trials += 1
        else:
            for i in range (startSlot, endSlot):
                farm[farm_row_tmp][i] = server_id
            break
                    
#     if server_id < 1:
#         print (farm[farm_row_tmp], server_slots)
    
    farm_row_tmp = (farm_row_tmp + 1) % (farm_rows)

Failed at server_id :  509 5.0
Failed at server_id :  521 2.0
Failed at server_id :  522 2.0
Failed at server_id :  523 2.0
Failed at server_id :  524 3.0
Failed at server_id :  525 3.0
Failed at server_id :  526 3.0
Failed at server_id :  527 3.0
Failed at server_id :  528 3.0
Failed at server_id :  529 3.0
Failed at server_id :  530 3.0
Failed at server_id :  531 4.0
Failed at server_id :  532 4.0
Failed at server_id :  533 4.0
Failed at server_id :  534 4.0
Failed at server_id :  535 4.0
Failed at server_id :  536 4.0
Failed at server_id :  537 4.0
Failed at server_id :  538 4.0
Failed at server_id :  539 4.0
Failed at server_id :  540 5.0
Failed at server_id :  541 5.0
Failed at server_id :  542 5.0
Failed at server_id :  543 5.0
Failed at server_id :  544 5.0
Failed at server_id :  549 1.0
Failed at server_id :  550 1.0
Failed at server_id :  551 1.0
Failed at server_id :  552 1.0
Failed at server_id :  553 1.0
Failed at server_id :  554 1.0
Failed at server_id :  555 1.0
Failed a

In [74]:
for each in farm:
    print (each)

[ 16  16  32  32  32  32  48  64  64  64  64  64  80  96  96  96  96 112
 112 112 112 112 128 128 144 144 144 144 160 176 176 176 192 192 192 192
 208 208 224 224 224 240 240 240 240 256 272 272 272 272 288 304 304 304
 320 320 320 320 320 336 336 336 352 352 352 352 368 384 384 384 400 400
 400 400 400 416 416 432 432 432 432 432 448 448 448 464 464 464 464 464
 480 495 495 495 495 516 516 548  -1   0]
[  1  17  17  17  33  33  33  33  49  65  65  65  65  65  81  97  97  97
  97 113 113 113 113 113 129 129 145 145 145 145 161 177 177 177 193 193
 193 193 209 209 225 225 225 241 241 241 241 257 273 273 273 273 289 305
 305 305 321 337 337 337 353 353 353 353 369 385 385 385 401 401 401 401
 401 417 417 417 433 433 433 433 433 449 449 449 465 465 465 465 465 481
 496 496 496 496 517 517 520 520  -1   0]
[  2  18  18  18  34  34  34  34  50  66  66  66  66  66  82  98  98  98
  98 114 114 114 114 114 130 130 146 146 146 146 162 178 178 178 194 194
 194 194 210 210 226 226 226 242 242 242

In [75]:
servers_info2[509:]

[[5.0, 40.0, 8.0],
 [1.0, 7.0, 7.0],
 [1.0, 7.0, 7.0],
 [1.0, 7.0, 7.0],
 [1.0, 7.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [2.0, 14.0, 7.0],
 [3.0, 21.0, 7.0],
 [3.0, 21.0, 7.0],
 [3.0, 21.0, 7.0],
 [3.0, 21.0, 7.0],
 [3.0, 21.0, 7.0],
 [3.0, 21.0, 7.0],
 [3.0, 21.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [4.0, 28.0, 7.0],
 [5.0, 35.0, 7.0],
 [5.0, 35.0, 7.0],
 [5.0, 35.0, 7.0],
 [5.0, 35.0, 7.0],
 [5.0, 35.0, 7.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [1.0, 6.0, 6.0],
 [2.0, 12.0, 6.0],
 [2.0, 12.0, 6.0],
 [2.0, 12.0, 6.0],
 [2.0, 12.0, 6.0],
 [2.0, 12.0, 6.0],
 [3.0, 18.0, 6.0],
 [3.0, 1